In [1]:
cd ~/src/Classes/Ansible/

/Users/rick446/src/Classes/Ansible


# Roles: _Really_ organizing your code

A *Role* is a directory structure organizing different parts of plays in ways that let them be included in other playbooks easily:

- **role_name**
  - files
  - templates
  - tasks
  - handlers
  - vars
  - defaults
  - meta

If you have a role defined, it cab be used in a play using the `roles:` key, e.g.:

```
---
- hosts: all
  roles:
    - my_custom_role
```

This means to look in `my_custom_role` files named `main.yml` and add them to the play. Particularly, 

- If `my_custom_role/tasks/main.yml` exists, it is added to the `tasks:` part of the play
- If `my_custom_role/handlers/main.yml` exists, it is added to the `handlers:` part of the play
- If `my_custom_role/vars/main.yml` exists, it is added to the `vars:` part of the play
- If `my_custom_role/defaults/main.yml` exists, it is added to the `vars:` part of the play (as super-low priority)
- If `my_custom_role/meta/main.yml` exists, it is added to the `roles:` part of the play (think "sub-roles")

# Ansible Galaxy: A shared repository of Roles

To use Ansible Galaxy, we first should set up a `roles_path` in `ansible.cfg`. Mine looks like this now:

```
[defaults]
# ...
roles_path = ./roles
```

Now we can browse the [Ansible Galaxy](https://galaxy.ansible.com/) website and use the `ansible-galaxy` command-line tool to install roles. 

## Examining a popular Galaxy role: nginx

In [ ]:
%%bash
ansible-galaxy install bennojoy.nginx
tree roles

 To use the role, we just need to add it to the `roles` key in our playbook. In this case, we'll also add some role variables to it:

In [ ]:
cat playbooks/setup-webservers.yaml

In [ ]:
%%bash
ansible-playbook playbooks/setup-webservers.yaml

Here is another playbook to set up per-user directories on the web hosts:

In [ ]:
cat playbooks/make-site.yaml

In [ ]:
%%bash
ansible-playbook playbooks/make-site.yaml

# Lab: Install a per-user nginx site

Ensure that your username and roles path is set in the ansible.cfg

```
ansible-galaxy install bennojoy.nginx

```

Create your own personal home page using the make-site playbook:

```
ansible-playbook playbooks/make-site.yaml
```

Access your website: http://arborian-01.class.arborian.com/~rick/ (use your username)